In [1]:
import pandas as pd
from itertools import combinations

In [2]:
ds = pd.read_parquet('ir_ds.pq')

In [3]:
ds['ir'] = ds.apply(lambda x: tuple(x['ir']),axis=1)

In [4]:
ds['opt']=ds.apply(lambda x: tuple(x['opt']),axis=1)

In [5]:
ds['build_options'].unique()

array(['-I.'], dtype=object)

In [6]:
ds.columns

Index(['kernelName', 'globalSize', 'localSize', 'transferredBytes',
       'transferTimeNs', 'kernelTimeNs', 'build_options', 'ir', 'opt',
       'kernelname', 'hostname', 'deviceName', 'benchmarkSuite',
       'benchmarkName', 'datasetName'],
      dtype='object')

In [7]:
group = ds.groupby(['benchmarkName','datasetName','benchmarkSuite','opt'])

In [8]:
def get_mean(grp):
    df = pd.DataFrame(columns=ds.columns)
    for g in grp:
        temp = g[1].iloc[0]
        temp['transferTimeNs'] = g[1]['transferTimeNs'].mean()
        temp['kernelTimeNs'] = g[1]['kernelTimeNs'].mean()
#         return temp
        df = df.append(temp,ignore_index=True)
    return df

In [9]:
df = get_mean(group)

In [10]:
group = df.groupby(['benchmarkName','datasetName','benchmarkSuite'])

In [11]:
def get_pair_data(grp):
    df = pd.DataFrame(columns=['ir1','globalSize1','localSize1','transferTimeNs1','kernelTimeNs1','ir2','globalSize2','localSize2','transferTimeNs2','kernelTimeNs2'])
    k=0
    for g in grp:
        temp = g[1].loc[g[1]['opt']==()]
        transferTimeNs = temp['transferTimeNs'].iloc[0]
        kernelTimeNs = temp['kernelTimeNs'].iloc[0]
        temp = g[1]
        temp['transferTimeNs']=temp['transferTimeNs']-transferTimeNs
        temp['kernelTimeNs']=temp['kernelTimeNs']-kernelTimeNs
        for comb in combinations(temp.index,2):
            first = temp.loc[comb[0]]
            second = temp.loc[comb[1]]
            df.loc[k]=[first['ir'],first['globalSize'],first['localSize'],first['transferTimeNs'],first['kernelTimeNs'],second['ir'],second['globalSize'],second['localSize'],second['transferTimeNs'],second['kernelTimeNs']]
            k+=1
    return df

In [12]:
d = get_pair_data(group)
d

/home/abhinav/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,ir1,globalSize1,localSize1,transferTimeNs1,kernelTimeNs1,ir2,globalSize2,localSize2,transferTimeNs2,kernelTimeNs2
0,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,0.000000e+00,0.000000e+00,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,-1.021967e+31,-1.312081e+23
1,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,0.000000e+00,0.000000e+00,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,-1.021967e+31,-1.312081e+23
2,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,-1.021967e+31,-1.312081e+23,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",22115328,128,-1.021967e+31,-1.312081e+23
3,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",28530688,8,0.000000e+00,0.000000e+00,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",28530688,8,-1.316338e+31,-1.275880e+27
4,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",29387776,128,0.000000e+00,0.000000e+00,"(-0.567818, 0.846908, 1.716297, 1.124501, -0.8...",29387776,128,-1.355856e+31,-1.735442e+23
...,...,...,...,...,...,...,...,...,...,...
1859,"(0.236931, 0.861191, 1.331397, 0.950226, -0.70...",24112128,512,0.000000e+00,0.000000e+00,"(0.236931, 0.861191, 1.331397, 0.950226, -0.70...",24112128,512,-2.224589e+15,-1.716158e+11
1860,"(0.236931, 0.861191, 1.331397, 0.950226, -0.70...",27850752,512,0.000000e+00,0.000000e+00,"(0.236931, 0.861191, 1.331397, 0.950226, -0.70...",27850752,512,-2.569504e+15,-1.961598e+11
1861,"(-0.348847, 0.915227, 1.825628, 1.322826, -0.6...",17167360,32,0.000000e+00,0.000000e+00,"(-0.348847, 0.915227, 1.825628, 1.322826, -0.6...",17167360,32,-1.584264e+15,-4.092637e+11
1862,"(-0.348847, 0.915227, 1.825628, 1.322826, -0.6...",17976320,256,0.000000e+00,0.000000e+00,"(-0.348847, 0.915227, 1.825628, 1.322826, -0.6...",17976320,256,-1.658848e+15,-2.865757e+11


In [6]:
import gpgpu_pb2

In [7]:
b = gpgpu_pb2.BenchmarkRun()